In [ ]:

#  Jupyter NoteBook to make a graph with (n) nodes/vertices of Label: Airport
#


#  Setup stuff: Connectivity

In [ ]:

import pandas as pd
   #
from katana import remote
from katana.remote import import_data
   #
from tabulate import tabulate


my_client = remote.Client()
   #
print("Katana Client handle: %s" % (str(my_client)) )


In [ ]:

NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


In [ ]:

#  If for whatever reason you wish to reset, this will delete our graph
#

for l_database in my_client.databases():
   for l_graph in my_client.get_database(name = l_database.name).graphs_in_database():
      if ((l_database.name == DB_NAME) and (l_graph.name == GRAPH_NAME)):   
         l_handle = my_client.get_database(name = l_database.name).get_graph_by_id(id = l_graph.graph_id)
         l_handle.delete()
         print("Graph deleted: %s" % (GRAPH_NAME) )

print("--")


In [ ]:

#  Create database

my_database = my_client.create_database(name = DB_NAME)

print(my_database.database_id)


In [ ]:

#  Create graph
#
my_graph = my_client.get_database(name=DB_NAME).create_graph(name=GRAPH_NAME, num_partitions=NUM_PARTITIONS)

print(my_graph)


In [ ]:

#  Connect to graph

my_graph, *_ = my_client.get_database(name=DB_NAME).find_graphs_by_name(GRAPH_NAME)

print(my_graph)


# Create: Vertices/nodes, edges ..

In [ ]:

#  Vertices/Nodes, Airports ..

df_airports = pd.DataFrame([
       #
   ["MKE", "Milwaukee"               , "Airport"],
   ["ORD", "Chicago O-Hare"          , "Airport"],
   ["SJC", "San Jose"                , "Airport"],
   ["DEN", "Denver"                  , "Airport"],
       #
   ], columns = ["airport_code", "airport_name", "LABEL"])

print(tabulate(df_airports, headers='keys', tablefmt='psql'))


In [ ]:
#  Create the edge, flights ..

df_flights = pd.DataFrame([
      #
   ["MKE", "ORD",   66, 1, "FLIES_TO" ],
   ["ORD", "MKE",   66, 1, "FLIES_TO" ],
      #
   ["ORD", "DEN",  886, 1, "FLIES_TO" ],
   ["DEN", "ORD",  886, 1, "FLIES_TO" ],
      #
   ["SJC", "DEN",  948, 1, "FLIES_TO" ],                           #  Notice SJC flies to/from Denver
   ["DEN", "SJC",  948, 1, "FLIES_TO" ],
      #
   ["SJC", "ORD", 1829, 1, "FLIES_TO" ],                           #  Notice SJC flies to ORD, but not ORD to SJC
      #
   ], columns = ["START_ID", "END_ID", "DISTANCE", "NUM_HOPS", "TYPE"])

print(tabulate(df_flights, headers='keys', tablefmt='psql'))


In [ ]:

# Import the 2 previously created Python DataFrames into KatanaGraph ..

with import_data.DataFrameImporter(my_graph) as df_importer:   
    
   df_importer.nodes_dataframe(df_airports,                    #  Aiport set of Nodes
      id_column             = "airport_code",
      id_space              = "Airport",  
      label                 = "Airport",  
      )
   
   df_importer.edges_dataframe(df_flights,                     #  Our Edge, specifying the relationship between Airport --> FLIES_TO --> Airport
      source_id_space       = "Airport", 
      destination_id_space  = "Airport",   
      source_column         = "START_ID",
      destination_column    = "END_ID",
      type                  = "FLIES_TO"
      )

   df_importer.insert()

print("--")


In [11]:

#  What's in the graph/nodes ..

l_result = my_graph.query_unpaginated("""
   MATCH (n) 
   RETURN n
   """)
print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+-----------------+-------------+-----------+------------------+------------------+--------------+
#     |    |           id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   | n datatype   |
#     |----+-----------------+-------------+-----------+------------------+------------------+--------------|
#     |  0 | 281474976710656 | ['Airport'] | Airport   | MKE              | Milwaukee        | node         |
#     |  1 | 281474976710657 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node         |
#     |  2 | 562949953421312 | ['Airport'] | Airport   | SJC              | San Jose         | node         |
#     |  3 | 562949953421313 | ['Airport'] | Airport   | DEN              | Denver           | node         |
#     +----+-----------------+-------------+-----------+------------------+------------------+--------------+


          0/? [?op/s]

+----+-----------------+-------------+-----------+------------------+------------------+--------------+
|    |           id(n) | labels(n)   | n.LABEL   | n.airport_code   | n.airport_name   | n datatype   |
|----+-----------------+-------------+-----------+------------------+------------------+--------------|
|  0 |               0 | ['Airport'] | Airport   | SJC              | San Jose         | node         |
|  1 | 281474976710656 | ['Airport'] | Airport   | DEN              | Denver           | node         |
|  2 | 562949953421313 | ['Airport'] | Airport   | ORD              | Chicago O-Hare   | node         |
|  3 | 562949953421312 | ['Airport'] | Airport   | MKE              | Milwaukee        | node         |
+----+-----------------+-------------+-----------+------------------+------------------+--------------+


In [ ]:

#  What's in the edge .. 

l_result = my_graph.query_unpaginated("""

   MATCH (n) - [r] -> (m)
   RETURN r
   
   """)

print(tabulate(l_result, headers='keys', tablefmt='psql'))

#  Sample output,
#
#     +----+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
#     |    |   id(endNode(r)) | labels(endNode(r))   |           id(r) | type(r)   |   r.DISTANCE |   r.NUM_HOPS | r.TYPE   |   id(startNode(r)) | labels(startNode(r))   | r datatype   |
#     |----+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------|
#     |  0 |  281474976710656 | ['Airport']          | 281474976710656 | FLIES_TO  |           66 |            1 | FLIES_TO |    281474976710657 | ['Airport']            | edge         |
#     |  1 |  562949953421313 | ['Airport']          | 281474976710658 | FLIES_TO  |          886 |            1 | FLIES_TO |    281474976710657 | ['Airport']            | edge         |
#     |  2 |  281474976710657 | ['Airport']          | 281474976710657 | FLIES_TO  |           66 |            1 | FLIES_TO |    281474976710656 | ['Airport']            | edge         |
#     |  3 |  562949953421313 | ['Airport']          | 562949953421313 | FLIES_TO  |          948 |            1 | FLIES_TO |    562949953421312 | ['Airport']            | edge         |
#     |  4 |  281474976710657 | ['Airport']          | 562949953421314 | FLIES_TO  |         1829 |            1 | FLIES_TO |    562949953421312 | ['Airport']            | edge         |
#     |  5 |  562949953421312 | ['Airport']          | 562949953421312 | FLIES_TO  |          948 |            1 | FLIES_TO |    562949953421313 | ['Airport']            | edge         |
#     |  6 |  281474976710657 | ['Airport']          | 562949953421315 | FLIES_TO  |          886 |            1 | FLIES_TO |    562949953421313 | ['Airport']            | edge         |
#     +----+------------------+----------------------+-----------------+-----------+--------------+--------------+----------+--------------------+------------------------+--------------+
